In [2]:
import pandas as pd
import numpy as np
import re
import random
import torch
import torch.nn as nn
import torch.optim as optim
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import StratifiedKFold, train_test_split
from sklearn.naive_bayes import MultinomialNB
from sklearn.svm import LinearSVC
from sklearn.ensemble import RandomForestClassifier, VotingClassifier
from xgboost import XGBClassifier
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from nltk.tokenize import word_tokenize
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Embedding, Conv1D, MaxPooling1D, GlobalMaxPooling1D, Dense, LSTM, GRU, Bidirectional
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.utils import to_categorical
import tensorflow as tf
import warnings
warnings.filterwarnings("ignore")
tf.get_logger().setLevel('ERROR')

# ---------------------------
# Step 1. Load Dataset
# ---------------------------
dataset_path = r"Processed_Causality_Dataset.csv"

print("=== Loading Dataset ===")
df = pd.read_csv(dataset_path)

# Extract raw features + labels
X_raw = df["Sentence"].astype(str)
y_raw = df["Causality_Label"]

# Encode labels
label_encoder = LabelEncoder()
y = label_encoder.fit_transform(y_raw)

# Train/test split (just for check; CV will use full training set)
X_train_raw, X_test_raw, y_train, y_test = train_test_split(
    X_raw, y, stratify=y, test_size=0.2, random_state=42
)

print(f"Dataset File: {dataset_path}")
print(f"Total Samples: {df.shape[0]}, Columns: {df.shape[1]}")
print(f"Train Split: {len(X_train_raw)} | Test Split: {len(X_test_raw)}")
print(f"Label Classes: {list(label_encoder.classes_)}\n")


# ---------------------------
# 3. Embedding Generators
# ---------------------------
def get_bow(X):
    return CountVectorizer().fit_transform(X).toarray()

def get_tfidf(X):
    return TfidfVectorizer().fit_transform(X).toarray()

def get_word2vec(X, size=100, window=5, min_count=2, sg=0):
    tokens = [nltk.word_tokenize(t) for t in X]
    model = Word2Vec(tokens, vector_size=size, window=window, min_count=min_count, workers=4, sg=sg)
    return np.array([np.mean([model.wv[w] for w in words if w in model.wv] or [np.zeros(size)], axis=0)
                     for words in tokens])

def get_fasttext(X, size=100, window=5, min_count=2):
    tokens = [nltk.word_tokenize(t) for t in X]
    model = FastText(tokens, vector_size=size, window=window, min_count=min_count, workers=4)
    return np.array([np.mean([model.wv[w] for w in words if w in model.wv] or [np.zeros(size)], axis=0)
                     for words in tokens])

def get_glove(X, glove_path="glove.6B.100d.txt", size=100):
    embeddings_index = {}
    with open(glove_path, encoding="utf8") as f:
        for line in tqdm(f, desc="Reading GloVe"):
            values = line.split()
            word = values[0]
            coefs = np.asarray(values[1:], dtype="float32")
            embeddings_index[word] = coefs
    
    tokens = [nltk.word_tokenize(t) for t in X]
    vectors = []
    for words in tokens:
        vectors.append(np.mean([embeddings_index[w] for w in words if w in embeddings_index] or [np.zeros(size)], axis=0))
    return np.array(vectors)

# ---------------------------
# 4. Classifiers
# ---------------------------
def build_voting_classifier():
    clf1 = SVC(kernel="linear", probability=True)
    clf2 = XGBClassifier(use_label_encoder=False, eval_metric="mlogloss")
    clf3 = RandomForestClassifier(n_estimators=100, random_state=42)
    clf4 = GaussianNB()
    ensemble = VotingClassifier(estimators=[
        ("svm", clf1), ("xgb", clf2), ("rf", clf3), ("nb", clf4)
    ], voting="soft")
    return ensemble

# ---------------------------
# 5. Cross Validation
# ---------------------------
def evaluate_embedding(X_emb, y, name):
    print(f"\n=== Training Embedding: {name} + Soft Voting Ensemble ===")
    skf = StratifiedKFold(n_splits=10, shuffle=True, random_state=42)

    accs, precs, recs, f1s = [], [], [], []

    for fold, (train_idx, test_idx) in enumerate(skf.split(X_emb, y), 1):
        X_train, X_test = X_emb[train_idx], X_emb[test_idx]
        y_train, y_test = y.iloc[train_idx], y.iloc[test_idx]

        clf = build_voting_classifier()
        clf.fit(X_train, y_train)
        y_pred = clf.predict(X_test)

        accs.append(accuracy_score(y_test, y_pred) * 100)
        precs.append(precision_score(y_test, y_pred, average="macro") * 100)
        recs.append(recall_score(y_test, y_pred, average="macro") * 100)
        f1s.append(f1_score(y_test, y_pred, average="macro") * 100)

        print(f"Fold {fold:02d} -> Accuracy: {accs[-1]:.2f} | Precision: {precs[-1]:.2f} | "
              f"Recall: {recs[-1]:.2f} | F1: {f1s[-1]:.2f}")


# ---------------------------
# 6. Run Experiments
# ---------------------------
embeddings = {
    "BoW": get_bow(X_processed),
    "TF-IDF": get_tfidf(X_processed),
    "Word2Vec-CBOW": get_word2vec(X_processed, sg=0),
    "Word2Vec-SkipGram": get_word2vec(X_processed, sg=1),
    "FastText": get_fasttext(X_processed),
    "GloVe": get_glove(X_processed, glove_path="glove.6B.100d.txt"),
}

for name, X_emb in embeddings.items():
    evaluate_embedding(np.array(X_emb), y_raw, name)


# ---------------------------
# Step 2: Summary Table
# ---------------------------
df_embed = pd.DataFrame(embedding_results, columns=[
    "Embedding", "Accuracy", "Precision", "Recall", "F1-Score"
])

print("=== Final Embedding-Level Results (Soft Voting, 10-Fold CV) ===\n")
print(df_embed.to_string(index=False))


=== Loading Dataset ===
Dataset File: Processed_Causality_Dataset.csv
Total Samples: 376, Columns: 2
Train Split: 300 | Test Split: 76
Label Classes: [0, 1]

=== Training Model: Word2Vec + Soft Voting ===
10-Fold CV -> Accuracy: 81.58 | Precision: 81.25 | Recall: 81.25 | F1: 81.25

=== Training Model: GloVe + Soft Voting ===
10-Fold CV -> Accuracy: 81.58 | Precision: 81.25 | Recall: 81.61 | F1: 81.37

=== Training Model: FastText + Soft Voting ===
10-Fold CV -> Accuracy: 82.89 | Precision: 82.97 | Recall: 82.06 | F1: 82.38

=== Training Model: BoW + Soft Voting ===
10-Fold CV -> Accuracy: 88.16 | Precision: 90.08 | Recall: 86.72 | F1: 87.53

=== Training Model: TF-IDF + Soft Voting ===
10-Fold CV -> Accuracy: 93.42 | Precision: 93.99 | Recall: 92.78 | F1: 93.22

=== Training Model: Skip-gram + Soft Voting ===
10-Fold CV -> Accuracy: 98.26 | Precision: 98.32 | Recall: 98.14 | F1: 98.53

=== Final Embedding-Level Results (Soft Voting, 10-Fold CV) ===

   Embedding  Accuracy  Precision  R